# Docker Swarm with Blinkt!

## Init Docker Swarm on Cluster

Zunächst die IP-Adresse feststellen.

In [8]:
getent hosts $(hostname)

127.0.1.1       cluster-00.rdf.loc cluster-00


Geht nicht, da hier nur die Einträge aus der `hosts`-Datei ausgelesen werden.

In [9]:
cat /etc/hosts

127.0.0.1 localhost
127.0.1.1 cluster-00.rdf.loc cluster-00
::1             localhost ip6-localhost ip6-loopback
fe00::0         ip6-localnet
ff00::0         ip6-mcastprefix
ff02::1         ip6-allnodes
ff02::2         ip6-allrouters


Besser die IP und Namen durch den DNS aus dem lokalen Netz verwenden.

Hier wird das Paket dnsutils nachinstalliert, falls nslookup nicht vorhanden ist.

In [10]:
nslookup $(hostname) || (sudo apt-get install -yes dnsutils && nslookup $(hostname))

Server:		192.168.56.47
Address:	192.168.56.47#53

Name:	cluster-00.fritz.box
Address: 192.168.56.192



In [11]:
FQDN=$(nslookup $(hostname) | grep Name | awk '{ print $2 }')
echo $FQDN

cluster-00.fritz.box


In [12]:
IP=$(getent hosts ${FQDN} | awk '{ print $1 }')
echo $IP

192.168.56.192


In [13]:
DOMAIN=$(getent hosts ${FQDN} | awk '{ print $2 }' | cut -d '.' -f2,3)
echo $DOMAIN

fritz.box


In [14]:
docker node ls || docker swarm init --advertise-addr ${IP}

Error response from daemon: This node is not a swarm manager. Use "docker swarm init" or "docker swarm join" to connect this node to swarm and try again.
Swarm initialized: current node (gg85b6mhkcn767gw5wn2b450e) is now a manager.

To add a worker to this swarm, run the following command:

    docker swarm join \
    --token SWMTKN-1-4ss54vka9dgkqjvf0x0amgwb4g7h7ylnhrsmxfe6r8s89a8u24-e9l3zf2rtwngdxezt3yv5sn67 \
    192.168.56.192:2377

To add a manager to this swarm, run 'docker swarm join-token manager' and follow the instructions.



In [15]:
JOIN_TOKEN=$(docker swarm join-token -q worker)
echo $JOIN_TOKEN

SWMTKN-1-4ss54vka9dgkqjvf0x0amgwb4g7h7ylnhrsmxfe6r8s89a8u24-e9l3zf2rtwngdxezt3yv5sn67


In [16]:
# Setup ssh-key for user if not exists
ls ~/.ssh/id_rsa || ansible -i localhost, -m shell -a 'ssh-keygen -b 2048 -t rsa -f ~/.ssh/id_rsa -q -N "" creates=~/.ssh/id_rsa' --connection=local localhost

/home/pi/.ssh/id_rsa


Den SSH Zugang vorbereiten. Zunächst aus `knwon_hosts` löschen. Dann wieder bekannt geben und den ssh-key übertragen.

In [17]:
for i in 01 02 03 04; do
    # Remove host key for ip
    CL_HOSTNAME=cluster-${i}
    echo ${CL_HOSTNAME}
    # Cluster Hostname only
    ssh-keygen -R ${CL_HOSTNAME}
    # Cluster FQDN
    ssh-keygen -R ${CL_HOSTNAME}.${DOMAIN}
    # IP Cluster 
    ssh-keygen -R $(getent hosts ${CL_HOSTNAME} | awk '{ print $1 }')
    # Add ip into known_hosts
    ssh-keyscan -H cluster-${i} >> ~/.ssh/known_hosts
    sshpass -p raspberry ssh-copy-id pi@cluster-${i}
done

cluster-01
# Host cluster-01 found: line 3 type RSA
# Host cluster-01 found: line 4 type ECDSA
# Host cluster-01 found: line 5 type ED25519
/home/pi/.ssh/known_hosts updated.
Original contents retained as /home/pi/.ssh/known_hosts.old
/home/pi/.ssh/known_hosts updated.
Original contents retained as /home/pi/.ssh/known_hosts.old
# Host 192.168.56.189 found: line 3 type ECDSA
/home/pi/.ssh/known_hosts updated.
Original contents retained as /home/pi/.ssh/known_hosts.old
# cluster-01 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
/usr/bin/ssh-copy-id: INFO: attempting to log in with the new key(s), to filter out any that are already installed

/usr/bin/ssh-copy-id: WARNING: All keys were skipped because they already exist on the remote system.

cluster-02
# Host cluster-02 found: line 3 type RSA
# Host cluster-02 found: line 4 type ECDSA
# Host cluster-02 found: line 5 type ED25519
/home/pi/.

Alle Clients als Worker dem Swarm beitreten.

In [18]:
for i in 01 02 03 04; do
    CL_HOSTNAME=cluster-${i}
    echo $CL_HOSTNAME
    CL_IP=$(getent hosts ${CL_HOSTNAME} | awk '{ print $1 }')
    echo $CL_IP
    ssh pi@cluster-${i} "docker swarm join --token $JOIN_TOKEN --advertise-addr ${CL_IP} ${IP}:2377"
done

cluster-01
192.168.56.189
This node joined a swarm as a worker.
cluster-02
192.168.56.191
This node joined a swarm as a worker.
cluster-03
192.168.56.187
This node joined a swarm as a worker.
cluster-04
192.168.56.196
This node joined a swarm as a worker.


In [19]:
docker node ls

ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
gg85b6mhkcn767gw5wn2b450e *   cluster-00          Ready               Active              Leader
i4oclfprt80sgrcgtmubwo0h2     cluster-02          Ready               Active              
nbwj4hqjnzos1gtgmrzez8nd3     cluster-04          Ready               Active              
pp20jv9n54gta0hh54ca0l3mj     cluster-03          Ready               Active              
wx23y4qs491snmxq14cw8pgbo     cluster-01          Ready               Active              


## Visualizer anschauen

Einen graphischen Output als Service starten.

In [20]:
docker service create \
  --name=visualizer \
  --publish=8000:8080/tcp \
  --constraint=node.role==manager \
  --mount=type=bind,src=/var/run/docker.sock,dst=/var/run/docker.sock \
  alexellis2/visualizer-arm

sqj3abx4oeyhbh5fngk8457ss
Since --detach=false was not specified, tasks will be created in the background.
In a future release, --detach=false will become the default.


In [21]:
echo "http://$(getent hosts cluster-00.fritz.box | awk '{ print $1 }'):8000"

http://192.168.56.192:8000


In [22]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
sqj3abx4oeyh        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp


Auf allen nodes den Monitor Dienst installieren. Dieser überwacht, wie viele whoami Container laufen.

Grün: Ein whoami-Container wird gestartet.

Rot: Ein whoami-Container stoppt.

Gelb: Für die Version 1.1.0

Blau: Für die Version 1.2.0

In [23]:
docker service create --name monitor --mode global \
  --restart-condition any --mount type=bind,src=/sys,dst=/sys \
  --mount type=bind,src=/var/run/docker.sock,dst=/var/run/docker.sock \
  stefanscherer/monitor:1.1.0

3e9my1weilpmm4afrvqze9vxh
Since --detach=false was not specified, tasks will be created in the background.
In a future release, --detach=false will become the default.


Es läuft noch kein whoami.

In [24]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
3e9my1weilpm        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
sqj3abx4oeyh        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp


In [25]:
docker service create --name whoami stefanscherer/whoami:1.1.0

ygzh4phxcp0i28dunm85enf7y
Since --detach=false was not specified, tasks will be created in the background.
In a future release, --detach=false will become the default.


In [ ]:
sleep 10

In [26]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
3e9my1weilpm        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
sqj3abx4oeyh        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
ygzh4phxcp0i        whoami              replicated          1/1                 stefanscherer/whoami:1.1.0         


In [27]:
docker service scale whoami=5

whoami scaled to 5


In [ ]:
sleep 30

In [28]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
3e9my1weilpm        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
sqj3abx4oeyh        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
ygzh4phxcp0i        whoami              replicated          5/5                 stefanscherer/whoami:1.1.0         


In [29]:
docker service scale whoami=15

whoami scaled to 15


In [ ]:
sleep 30

In [30]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
3e9my1weilpm        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
sqj3abx4oeyh        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
ygzh4phxcp0i        whoami              replicated          15/15               stefanscherer/whoami:1.1.0         


In [ ]:
docker service scale whoami=40

In [ ]:
sleep 45

In [31]:
docker service update --update-parallelism 5 \
  --image stefanscherer/whoami:1.2.0 whoami

whoami
Since --detach=false was not specified, tasks will be updated in the background.
In a future release, --detach=false will become the default.


In [ ]:
sleep 45

In [32]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
3e9my1weilpm        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
sqj3abx4oeyh        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
ygzh4phxcp0i        whoami              replicated          15/15               stefanscherer/whoami:1.2.0         


In [33]:
docker service scale whoami=1

whoami scaled to 1


In [ ]:
sleep 15

In [34]:
docker service scale whoami=40

whoami scaled to 40


In [ ]:
sleep 45

In [35]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
3e9my1weilpm        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
sqj3abx4oeyh        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
ygzh4phxcp0i        whoami              replicated          40/40               stefanscherer/whoami:1.2.0         


In [36]:
docker service update --update-parallelism 5 \
  --image stefanscherer/whoami:1.1.0 whoami

whoami
Since --detach=false was not specified, tasks will be updated in the background.
In a future release, --detach=false will become the default.


In [ ]:
sleep 45

In [37]:
docker service update --update-parallelism 5 \
  --image stefanscherer/whoami:1.2.0 whoami

whoami
Since --detach=false was not specified, tasks will be updated in the background.
In a future release, --detach=false will become the default.


In [ ]:
sleep 45

In [38]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
3e9my1weilpm        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
sqj3abx4oeyh        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
ygzh4phxcp0i        whoami              replicated          40/40               stefanscherer/whoami:1.2.0         


In [ ]:
END=10
for i in $(seq 1 $END); do
    docker service scale whoami=1 
    docker service update --update-parallelism 5 \
        --image stefanscherer/whoami:1.1.0 whoami --detach=false
    sleep 5
    docker service scale whoami=15
    sleep 10
    docker service update --update-parallelism 5 \
        --image stefanscherer/whoami:1.2.0 whoami --detach=false
    sleep 15
    docker service scale whoami=40
    sleep 15
    docker service update --update-parallelism 5 \
        --image stefanscherer/whoami:1.1.0 whoami --detach=false
    sleep 30
    docker service update --update-parallelism 5 \
        --image stefanscherer/whoami:1.2.0 whoami --detach=false
    sleep 30
done

whoami scaled to 1
whoami

all progress: 0 out of 1 tasks 
all progress: 1 out of 1 tasks 
whoami scaled to 15ds to verify that tasks are stable... 
whoami

all progress: 0 out of 15 tasks 
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
ll progress: 15 out of 15 tasks 
whoami scaled to 40ds to verify that tasks are stable... 
whoami



# Aufräumen

In [1]:
docker service rm whoami

whoami


In [2]:
docker service rm monitor

monitor


In [3]:
docker node ls

ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
11qekn6q3wjrbdaij9lg0ehw9     cluster-01          Ready               Active              
7xy1j4849hfhdt26nmmzxcuhs     cluster-04          Ready               Active              
k0liool22gqh16k0vo2hinn6y     cluster-03          Ready               Active              
ndxbhoyubmotz8c3cw6nm7tra     cluster-02          Ready               Active              
qanlrp0vw1g13p1zxxsow0e64 *   cluster-00          Ready               Active              Leader


In [4]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
1t8mxh50pg49        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp


In [5]:
for i in 01 02 03 04; do
    ssh pi@cluster-${i} "docker swarm leave --force"
done

Node left the swarm.
Node left the swarm.
Node left the swarm.
Node left the swarm.


In [6]:
docker service rm visualizer

visualizer


In [7]:
docker swarm leave --force

Node left the swarm.
